# CHPL Bulk Patron Load -- Sierra REST API

The purpose of this notebook is to create patron records in bulk from an Excel spreadsheet

In [1]:
!pip install -U sierra-ils-utils --quiet
!pip install -U pandas --quiet
!pip install -U openpyxl --quiet
!pwd

/home/ray/Documents/jupyter/sierra-ils-utils/bulk-patron-load


In [2]:
import asyncio
import json
import logging
import pandas as pd
import re
from sierra_ils_utils import SierraAPI

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    filename='patron_updates.log'
)

# LOADING config
with open('/home/ray/.config/sierra/api-prod.json') as f:
    config = json.load(f)

# !wget 
from chpl_patron_patch import StudentNew

In [3]:
# configure the REST API client
client = SierraAPI(
    config.get('sierra_api_base_url'),
    config.get('sierra_api_key'),
    config.get('sierra_api_secret')
)

response = client.request('GET', 'info/token')
response.raise_for_status()

<Response [200 200]>

In [4]:
df = pd.read_excel('./Northwest Local Student Data January 2025.xlsx')

students = []
for i, row in df.iterrows():
    student = StudentNew(
        last_name=row.get('Student Last Name'),
        first_name=row.get('Student First Name'),
        barcode=row.get('Card Number'),
        student_id=row.get('Student ID'),
        school_district=row.get('School District Name'),
        pin=row.get('pin'),
        school=row.get('School Name'),
        birth_date=row.get('Birth Date (MMDDYYYY)'),
        phone_number=row.get('Home Phone Number'),
        home_legal_address=row.get('Legal Home Address (no PO Box)'),
        home_legal_address_city=row.get('Legal Home Address City'),
        home_legal_address_state=row.get('Legal Home Address State'),
        home_legal_address_zip=row.get('Legal Home Address Zip Code'),
        # notice_pref=row.get('')
        email_address=row.get('E-mail Address'),
        home_library_code=row.get('Home Library Code'),
        patron_agency=row.get('Patron Agency Num'),
        alt_id=row.get('Alt ID (optional)'),
    )
    students.append(student)

In [ ]:
# students[0].patron_data

In [7]:
import asyncio

async def create_patron(student_obj):
    """
    Returns a tuple:    
      (barcode, record_number or None)
    """
    try:
        response = await client.async_request(
            'POST',
            'patrons/',
            json=student_obj.patron_data
        )
        response.raise_for_status()
        logging.info(response.text)

        data = response.json()
        link = data.get("link", "")
        match = re.search(r'/patrons/(\d+)', link)
        if match:
            record_number = match.group(1)
            logging.info(f"Created patron for barcode={student_obj.barcode} record={record_number}")
            return (student_obj.barcode, record_number)
        else:
            logging.warning(f"Could not parse record number from response: {data}")
            return (student_obj.barcode, None)

    except Exception as e:
        logging.error(f"Failed to create patron for barcode={student_obj.barcode}: {e}")
        return (student_obj.barcode, None)

async def process_in_batches(students, batch_size=25):
    """Process creating patrons in chunks of `batch_size`."""
    all_results = []
    # Loop through students in increments of `batch_size`
    for start in range(0, len(students), batch_size):
        batch = students[start:start + batch_size]
        # Send these 25 requests in parallel
        chunk_results = await asyncio.gather(
            *(create_patron(s) for s in batch),
            return_exceptions=True
        )
        all_results.extend(chunk_results)
    return all_results

# Actually run the processing
results = await process_in_batches(students, batch_size=25)

# Build and merge into the DataFrame
results_df = pd.DataFrame(results, columns=["barcode", "record_number"])
# Make sure types match before merging
df["Card Number"] = df["Card Number"].astype(str)
results_df["barcode"] = results_df["barcode"].astype(str)

df_merged = df.merge(
    results_df, 
    how='left', 
    left_on="Card Number", 
    right_on="barcode"
)
df_merged.drop(columns=["barcode"], inplace=True)
df_merged.to_excel('output.xlsx')  # ouput to excel

for r in results:
    if isinstance(r, Exception):
        logging.error(f"Gathered exception: {r}")